In [176]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn import preprocessing

In [177]:
users = pd.read_pickle('Users.pkl')

In [178]:
# Cargar Modelo
model = CatBoostClassifier().load_model("modelo")

In [179]:
users.columns

Index(['id', 'screen_name', 'complete_name', 'tweet_count', 'description',
       'location', 'birthdate', 'url', 'listed_count', 'created_at',
       'crawled_at', 'favs_count', 'followers_count', 'following_count',
       'verified'],
      dtype='object')

In [180]:
usuarios = users[['id','screen_name','complete_name']]

In [181]:
users.crawled_at = pd.to_datetime(users.crawled_at).dt.floor('S') # Corregir formato timestamp
users.created_at = pd.to_datetime(users.created_at).dt.floor('S') # Corregir formato timestamp
users.description = users.description.notna()  # Convertir a Dummy
users.screen_name = users.screen_name.notna()  # Convertir a Dummy
users.location = users.location.notna()  # Convertir a Dummy

In [182]:
# Favourites Count = The number of Tweets this user has liked in the account’s lifetime
# Transformar en ratio sobre tiempo de existencia
users.loc[:,'favs_tiempo'] = users.favs_count.astype(int) / ((pd.to_datetime(users.crawled_at,errors='coerce').astype(int) -  pd.to_datetime(users.created_at,errors='coerce').astype(int))/10000000000000000)
# Eliminar columna original
users.drop(columns='favs_count',inplace=True)

In [183]:
# Statuses Count = The number of Tweets this user has liked in the account’s lifetime
# Transformar en ratio sobre tiempo de existencia
users.loc[:,'statuses_tiempo'] = users.tweet_count.astype(int) / ((pd.to_datetime(users.crawled_at,errors='coerce').astype(int) -  pd.to_datetime(users.created_at,errors='coerce').astype(int))/10000000000000000)
# Eliminar columna original
users.drop(columns='tweet_count',inplace=True)

In [184]:
# Followers Count = The number of Tweets this user has liked in the account’s lifetime
# Transformar en ratio sobre tiempo de existencia
users.loc[:,'followers_tiempo'] = users.followers_count.astype(int) / ((pd.to_datetime(users.crawled_at,errors='coerce').astype(int) -  pd.to_datetime(users.created_at,errors='coerce').astype(int))/10000000000000000)
# Eliminar columna original
users.drop(columns='followers_count',inplace=True)

In [185]:
# Friends Count = The number of Tweets this user has liked in the account’s lifetime
# Transformar en ratio sobre tiempo de existencia
users.loc[:,'friends_tiempo'] = users.following_count.astype(int) / ((pd.to_datetime(users.crawled_at,errors='coerce').astype(int) -  pd.to_datetime(users.created_at,errors='coerce').astype(int))/10000000000000000)
# Eliminar columna original
users.drop(columns='following_count',inplace=True)

In [186]:
# Quedarnos con las columnas necesarias
users = users[['screen_name', 'description', 'location', 'listed_count', 'statuses_tiempo', 'favs_tiempo','followers_tiempo', 'friends_tiempo']]

In [187]:
# Preparar datos para aplicar el modelo
atributos = users.columns
to_predict = preprocessing.scale(users, axis = 0)

In [188]:
# Predecir
predicciones = model.predict(to_predict)

In [189]:
pd.DataFrame(predicciones)[0].value_counts()

1.0    2306
0.0    1192
Name: 0, dtype: int64